In [2]:
# stretching the cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ignoring the warnings
import warnings
warnings.filterwarnings('ignore', '.*do not.*',)

# cuda settings
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.50)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# take all the images and labels addresses
# make a function to convert the images into arrays
# create features from the arrays
# create tfrecords from the features

### Listing the images and labels

In [ ]:
import glob
train_addrs = glob.glob("coco-animals/train/*/*.jpg")
lbl_map = {"sheep":0,"cat":1,"dog":2,"horse":3,"bird":4,"giraffe":5,"zebra":6,"bear":7}
train_labels = []
for addrs in train_addrs:
    addrs = addrs.split("/")
    train_labels.append(lbl_map[addrs[2]])
assert len(train_addrs)==len(train_labels)    

In [ ]:
import glob
valid_addrs = glob.glob("coco-animals/val/*/*.jpg")
lbl_map = {"sheep":0,"cat":1,"dog":2,"horse":3,"bird":4,"giraffe":5,"zebra":6,"bear":7}
valid_labels = []
for addrs in valid_addrs:
    addrs = addrs.split("/")
    valid_labels.append(lbl_map[addrs[2]])
assert len(valid_addrs)==len(valid_labels)  

In [ ]:
count={}
for i in train_labels:
    x = count.setdefault(i,0)
    count[i] = x+1

In [ ]:
count

### Function to convert the images to arrays

In [ ]:
import cv2 
import numpy as np
def load_image(addr):
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

### Writing to tfrecords

In [ ]:
train_filename = 'train.tfrecords'
writer = tf.python_io.TFRecordWriter(train_filename)

for img_addrs,label in zip(train_addrs,train_labels):
    img = load_image(img_addrs)
    
    feature = {"image":_bytes_feature(tf.compat.as_bytes(img.tostring())),"label":_int64_feature(label)}
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())
writer.close()

In [ ]:
train_filename = 'valid.tfrecords'
writer = tf.python_io.TFRecordWriter(train_filename)

for img_addrs,label in zip(valid_addrs,valid_labels):
    img = load_image(img_addrs)
    
    feature = {"image":_bytes_feature(tf.compat.as_bytes(img.tostring())),"label":_int64_feature(label)}
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())
writer.close()

### Reading the tfrecords

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
data_path = "train.tfrecords"
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    
    feature = {'image':tf.FixedLenFeature([],tf.string), 'label':tf.FixedLenFeature([],tf.int64)}
    filename_queue = tf.train.string_input_producer([data_path],num_epochs=1)
    
    reader = tf.TFRecordReader()
    _,serialzed_example = reader.read(filename_queue)
    
    features = tf.parse_single_example(serialzed_example,features=feature)
    
    image = tf.decode_raw(features["image"],tf.float32)
    label = tf.cast(features["label"],tf.int32)
    
    image = tf.reshape(image,[224,224,3])
    
    images,labels = tf.train.shuffle_batch([image,label],batch_size=20,capacity=30,num_threads=1,min_after_dequeue=10)
    
    init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in range(1):
        img,lbl = sess.run([images,labels])
        print(img.shape,lbl.shape)
#         img = img.astype(np.uint8)
        
#         for j in range(10):
#             plt.subplot(4,3,j+1)
#             plt.imshow(img[j,...])
#             plt.title(lbl[j])
#         plt.show()
        
    coord.request_stop()

    coord.join(threads)
    sess.close()

(20, 224, 224, 3) (20,)


In [1]:
class Data:
    
    def __init__(self,image_size,num_classes,data_path="train.tfrecords",num_epochs=1,num_channels=3,batch_size=50,capacity=100,num_threads=1):
        self.data_path = data_path
        self.num_epochs = num_epochs
        self.image_size - image_size
        self.num_channels = num_channels
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.capacity = capacity
        self.num_threads = num_threads
    
    
    def read(self):
        feature = {'image':tf.FixedLenFeature([],tf.string), 'label':tf.FixedLenFeature([],tf.int64)}
        filename_queue = tf.train.string_input_producer([self.data_path],num_epochs=num_epochs)

        reader = tf.TFRecordReader()
        _,serialzed_example = reader.read(filename_queue)

        features = tf.parse_single_example(serialzed_example,features=feature)

        image = tf.decode_raw(features["image"],tf.float32)
        label = tf.cast(features["label"],tf.int32)

        image = tf.reshape(image,[self.image_size,self.image_size,self.num_channels])
        label = tf.one_hot(label,depth=self.num_classes)
        images,labels = tf.train.shuffle_batch([image,label],
                                               batch_size=self.batch_size,
                                               capacity=self.capacity,
                                               num_threads=self.num_threads,
                                               min_after_dequeue=10)

        return (images,labels)